In [3]:
import torch
import ray_rendering

In [4]:
R = 2.
c2w = ray_rendering.pose_spherical(90. + 10 + 45, -30., R)
N_samples = 16
N_samples_2 = 16
H = 32
W = H
focal = H * .9
near, far = R-1, R+1
rays = ray_rendering.get_rays(H, W, focal, c2w[:3,:4])

In [5]:
rays.shape

torch.Size([2, 32, 32, 3])

In [6]:
rays_o, rays_d = rays[0], rays[1]

In [36]:
# Compute 3D query points
z_vals = torch.linspace(near, far, N_samples)
pts = rays_o[...,None,:] + rays_d[...,None,:] * z_vals[...,:,None]

In [8]:
pts_flat = pts.reshape(-1, 3).numpy()

In [ ]:
import plotly.graph_objects as go

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,

    xaxis= go.layout.XAxis(linecolor = 'black',
                          linewidth = 1,
                          mirror = True),

    yaxis= go.layout.YAxis(linecolor = 'black',
                          linewidth = 1,
                          mirror = True),

    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad = 4
    )
)

fig = go.Figure(
    data=[
        go.Scatter3d(x=pts_flat[:,0], y=pts_flat[:,1], z=pts_flat[:,2], 
                     mode='markers',
                     marker=dict(
                        size=4,
                        opacity=0.8
                    )
        )
    ],
    layout = layout
)

fig.show()

### Properties

In [29]:
norm = rays_d.norm(dim=-1)

In [34]:
for i in range(16):
    print(norm[-i,-i], norm[i,i], norm[-i,i], norm[i,-i])

tensor(1.2717) tensor(1.2717) tensor(1.2717) tensor(1.2717)
tensor(1.2420) tensor(1.2420) tensor(1.2420) tensor(1.2420)
tensor(1.2135) tensor(1.2135) tensor(1.2135) tensor(1.2135)
tensor(1.1864) tensor(1.1864) tensor(1.1864) tensor(1.1864)
tensor(1.1607) tensor(1.1607) tensor(1.1607) tensor(1.1607)
tensor(1.1366) tensor(1.1366) tensor(1.1366) tensor(1.1366)
tensor(1.1141) tensor(1.1141) tensor(1.1141) tensor(1.1141)
tensor(1.0933) tensor(1.0933) tensor(1.0933) tensor(1.0933)
tensor(1.0744) tensor(1.0744) tensor(1.0744) tensor(1.0744)
tensor(1.0574) tensor(1.0574) tensor(1.0574) tensor(1.0574)
tensor(1.0425) tensor(1.0425) tensor(1.0425) tensor(1.0425)
tensor(1.0297) tensor(1.0297) tensor(1.0297) tensor(1.0297)
tensor(1.0191) tensor(1.0191) tensor(1.0191) tensor(1.0191)
tensor(1.0108) tensor(1.0108) tensor(1.0108) tensor(1.0108)
tensor(1.0048) tensor(1.0048) tensor(1.0048) tensor(1.0048)
tensor(1.0012) tensor(1.0012) tensor(1.0012) tensor(1.0012)


In [54]:
lv = 1
for i in range(1,4):
    print(torch.stack([pts[-i,-i,lv], pts[i,i,lv], pts[-i,i,lv], pts[i,-i,lv]], 0))

tensor([[-1.0833, -0.0779, -0.5180],
        [ 0.2223,  0.9445, -0.7116],
        [-0.1163, -0.0779, -1.1952],
        [-0.7447,  0.9445, -0.0345]])
tensor([[-1.0398, -0.0438, -0.5245],
        [ 0.1788,  0.9104, -0.7052],
        [-0.1372, -0.0438, -1.1565],
        [-0.7238,  0.9104, -0.0732]])
tensor([[-0.9963, -0.0097, -0.5309],
        [ 0.1353,  0.8764, -0.6987],
        [-0.1582, -0.0097, -1.1178],
        [-0.7028,  0.8764, -0.1119]])


In [45]:
pts[:,:,0,1]

tensor([[0.9811, 0.9811, 0.9811,  ..., 0.9811, 0.9811, 0.9811],
        [0.9511, 0.9511, 0.9511,  ..., 0.9511, 0.9511, 0.9511],
        [0.9210, 0.9210, 0.9210,  ..., 0.9210, 0.9210, 0.9210],
        ...,
        [0.1091, 0.1091, 0.1091,  ..., 0.1091, 0.1091, 0.1091],
        [0.0790, 0.0790, 0.0790,  ..., 0.0790, 0.0790, 0.0790],
        [0.0489, 0.0489, 0.0489,  ..., 0.0489, 0.0489, 0.0489]])

In [48]:
rays_d[:,:,1]

tensor([[-0.0189, -0.0189, -0.0189,  ..., -0.0189, -0.0189, -0.0189],
        [-0.0489, -0.0489, -0.0489,  ..., -0.0489, -0.0489, -0.0489],
        [-0.0790, -0.0790, -0.0790,  ..., -0.0790, -0.0790, -0.0790],
        ...,
        [-0.8909, -0.8909, -0.8909,  ..., -0.8909, -0.8909, -0.8909],
        [-0.9210, -0.9210, -0.9210,  ..., -0.9210, -0.9210, -0.9210],
        [-0.9511, -0.9511, -0.9511,  ..., -0.9511, -0.9511, -0.9511]])

Coordinate transformation

$$
[x_w,y_w,z_w] = T_{c2w} \cdot [x_c,y_c,z_c]^T
$$

In [47]:
c2w[:3,:3]

tensor([[-0.8192,  0.2868, -0.4967],
        [ 0.0000,  0.8660,  0.5000],
        [ 0.5736,  0.4096, -0.7094]])

In [50]:
j, i = torch.meshgrid(torch.arange(H), torch.arange(W))
dirs = torch.stack([(i-W*.5)/focal, -(j-H*.5)/focal, -torch.ones_like(i)], -1)

In [52]:
(dirs.unsqueeze(-2) * c2w[:3,:3]).shape

torch.Size([32, 32, 3, 3])

Transformation Matrix
```python
trans_t = lambda t : torch.tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=torch.float32)

rot_phi = lambda phi : torch.tensor([
    [1,0,0,0],
    [0,torch.cos(phi),-torch.sin(phi),0],
    [0,torch.sin(phi), torch.cos(phi),0],
    [0,0,0,1],
], dtype=torch.float32)

rot_theta = lambda th : torch.tensor([
    [torch.cos(th),0,-torch.sin(th),0],
    [0,1,0,0],
    [torch.sin(th),0, torch.cos(th),0],
    [0,0,0,1],
], dtype=torch.float32)

def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(torch.tensor(phi/180.*np.pi)) @ c2w
    c2w = rot_theta(torch.tensor(theta/180.*np.pi)) @ c2w
    # c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w
```

$$
\begin{bmatrix}
\cos\theta&0&-\sin\theta& 0\\
0 & 1 & 0 & 0\\
\sin\theta & 0 & \cos\theta & 0\\
0 & 0 & 0 & 1
\end{bmatrix}
\cdot
\begin{bmatrix}
1&0&0&0\\
0&\cos\phi&-\sin\phi&0\\
0&\sin\phi& \cos\phi&0\\
0&0&0&1
\end{bmatrix}
\cdot
\begin{bmatrix}
1 & 0 & 0 &0 \\
0 & 1 & 0 &0 \\
0 & 0 & 1 &t \\
0 & 0 & 0 &1
\end{bmatrix}
=
\begin{bmatrix}
\cos\theta & -\sin\theta\cos\phi & -\sin\theta\cos\phi & -t\sin\theta\cos\phi \\
0 & \cos\phi & -\sin\phi &-t\sin\phi \\
\sin\theta & \cos\theta\sin\phi & \cos\theta\cos\phi &t\cos\theta\cos\phi \\
0 & 0 & 0 &1
\end{bmatrix}
$$

**Observation:** y coordinate has the inherent **locality**!